In [3]:
const string PostsIndex = "posts";

#!import ./setup-clients.ipynb

# Index Management

### Create Index

In [2]:
var createResponse = await _client.Indices.CreateAsync(
    PostsIndex, c => c.Map<BlogPost>(m => m.AutoMap()));

DumpResponse(createResponse);

### Get Index Info

In [3]:
var dataIndexResponse = await client.Indices.GetAsync(PostsIndex);
var dataIndex = dataIndexResponse.Indices.First().Value;

DumpResponse(dataIndexResponse);
// display(dataIndex.Mappings.Properties);

### Delete Index

In [9]:
var response = await client.Indices.DeleteAsync(PostsIndex);
DumpResponse(response);

### Seed Data

In [4]:
Bogus.Randomizer.Seed = new Random(123);

var faker = new Bogus.Faker<BlogPost>()
    .RuleFor(p => p.Id, f => f.Random.Number(int.MaxValue))
    .RuleFor(p => p.Title, f => f.Commerce.ProductName())
    .RuleFor(p => p.Description, f => f.Lorem.Paragraph())
    .RuleFor(p => p.Author, f => f.Name.FullName())
    .RuleFor(p => p.CreatedAt, f => f.Date.Past());

var posts = faker.Generate(1000);

In [ ]:
using Microsoft.Data.Analysis;

var ids = new Int32DataFrameColumn("Id", posts.Select(p => p.Id));
var createdAt = new DateTimeDataFrameColumn("CreatedAt", posts.Select(p => p.CreatedAt));
var titles = new StringDataFrameColumn("Title", posts.Select(p => p.Title));
var descriptions = new StringDataFrameColumn("Description", posts.Select(p => p.Description));
var authors = new StringDataFrameColumn("Author", posts.Select(p => p.Author));
var dataFrame = new DataFrame(ids, createdAt, titles, descriptions, authors);

display(dataFrame);

In [5]:
var bulkResponse = await _client.BulkAsync(b => b
    .Index(PostsIndex)
    .IndexMany(posts)
);

display(bulkResponse.Errors);

In [ ]:
DumpResponse(bulkResponse);